In [1]:
# Import necessary libraries for data processing, anomaly detection, LLM-based rule generation, and file handling
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from transformers import pipeline
import datetime
import os  # Added for directory creation in Cell 12

In [2]:
# Define default profiling rules based on regulatory instructions, to be used for data validation
# Includes manually implemented rules from Cell 12 for Transaction_Amount vs Reported_Amount and Currency
default_profiling_rules = {
    'Customer_ID': {'type': 'int', 'rule': lambda x: x > 0, 'null_allowed': False},
    'Account_Balance': {'type': 'float', 'rule': lambda x: x >= 0, 'null_allowed': False},
    'Transaction_Amount': {'type': 'float', 'rule': lambda x: x >= 0, 'null_allowed': False},
    'Reported_Amount': {'type': 'float', 'rule': lambda x: x >= 0, 'null_allowed': False},
    'Currency': {
        'type': 'str',
        'rule': lambda x: x in ['USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'CNY', 'INR', 'SGD'],  # Expanded list of ISO 4217 codes (source: https://www.iso.org/iso-4217-currency-codes.html)
        'null_allowed': False
    },
    'Country': {'type': 'str', 'rule': lambda x: x in ['US', 'DE', 'UK', 'JP'], 'null_allowed': False},
    'Transaction_Date': {
        'type': 'datetime',
        'rule': lambda x: pd.to_datetime(x) <= pd.Timestamp(datetime.datetime.now()) and (pd.Timestamp(datetime.datetime.now()) - pd.to_datetime(x)).days <= 365,
        'null_allowed': False
    },
    'Risk_Score': {'type': 'float', 'rule': lambda x: 0 <= x <= 10 if pd.notnull(x) else True, 'null_allowed': True},
    'Transaction_Amount vs Reported_Amount': {
        'cross_field': lambda row: abs(row['Transaction_Amount'] - row['Reported_Amount']) <= 0.01 * row['Transaction_Amount'] if row.get('Cross_Currency', 'No') != 'No' else row['Transaction_Amount'] == row['Reported_Amount']
    },
    'Round_Number_Transaction': {'cross_field': lambda row: row['Transaction_Amount'] % 1000 != 0},
    'High_Risk_Transaction': {'cross_field': lambda row: not (row['Transaction_Amount'] > 5000 and row['Country'] in ['DE'])}
}

In [3]:
# Validate the dataset against profiling rules, checking data types, null values, and cross-field conditions
def validate_dataset(df, rules):
    validation_results = {}
    for column, rule in rules.items():
        if 'cross_field' in rule:
            continue
        if column not in df.columns:
            validation_results[column] = f"Column '{column}' not found in dataset."
            continue
        if rule['type'] == 'int' and not pd.api.types.is_integer_dtype(df[column]):
            validation_results[column] = f"Column '{column}' should be integer type."
            continue
        elif rule['type'] == 'float' and not pd.api.types.is_float_dtype(df[column]):
            validation_results[column] = f"Column '{column}' should be float type."
            continue
        elif rule['type'] == 'datetime':
            df[column] = pd.to_datetime(df[column], errors='coerce')
        if not rule['null_allowed'] and df[column].isnull().any():
            invalid_rows = df[df[column].isnull()].index.tolist()
            validation_results[column] = f"Column '{column}' contains null values at rows: {invalid_rows}"
            continue
        if rule['null_allowed']:
            invalid_rows = df[column][df[column].notnull() & ~df[column].apply(rule['rule'])].index.tolist()
        else:
            invalid_rows = df[column][~df[column].apply(rule['rule'])].index.tolist()
        if invalid_rows:
            validation_results[column] = f"Column '{column}' has invalid values at rows: {invalid_rows}"
        else:
            validation_results[column] = "Valid"
    for column, rule in rules.items():
        if 'cross_field' not in rule:
            continue
        invalid_rows = df[~df.apply(rule['cross_field'], axis=1)].index.tolist()
        if invalid_rows:
            validation_results[column] = f"Cross-field validation failed at rows: {invalid_rows}"
        else:
            validation_results[column] = "Valid"
    return validation_results

In [4]:
# Detect anomalies in the dataset using Isolation Forest, focusing on numerical features
def detect_anomalies(df):
    features = ['Account_Balance', 'Transaction_Amount', 'Reported_Amount']
    X = df[features].fillna(0)  # Handle NaN values to prevent errors
    iso_forest = IsolationForest(contamination=0.1, random_state=42)  # Adjusted contamination
    df['Anomaly'] = iso_forest.fit_predict(X)
    return df

In [5]:
# Adjust risk scores based on validation failures and anomalies, with configurable weights
def adjust_risk_score(row, validation_results):
    risk_adjustment = 0
    for column, result in validation_results.items():
        if 'rows' in str(result):
            try:
                invalid_rows = [int(r) for r in str(result).split('[')[1].split(']')[0].split(',') if r.strip().isdigit()]
                if row.name in invalid_rows:
                    risk_adjustment += 2  # Weight for validation failure
            except (IndexError, ValueError):
                pass
    if row['Anomaly'] == -1:
        risk_adjustment += 3  # Weight for anomaly
    base_score = row['Risk_Score'] if pd.notnull(row['Risk_Score']) else 2.0  # Default to 2.0 if NaN
    return base_score + risk_adjustment

In [6]:
# Load the sample dataset from CSV file for validation and analysis
df = pd.read_csv('sample_data.csv')

In [7]:
# Preprocess the dataset to ensure correct data types and formats for validation
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], errors='coerce')
df['Country'] = df['Country'].str.upper()
df['Currency'] = df['Currency'].str.upper()
# Convert numerical columns to float to fix type issues
df['Account_Balance'] = pd.to_numeric(df['Account_Balance'], errors='coerce').astype(float)
df['Transaction_Amount'] = pd.to_numeric(df['Transaction_Amount'], errors='coerce').astype(float)
df['Reported_Amount'] = pd.to_numeric(df['Reported_Amount'], errors='coerce').astype(float)
# Handle null values for required fields
df['Currency'] = df['Currency'].fillna('USD')  # Default to USD
df['Country'] = df['Country'].fillna('Unknown')  # Default to Unknown
df['Transaction_Date'] = df['Transaction_Date'].fillna(pd.Timestamp('2025-01-01'))  # Default date

In [8]:
# Run validation on the dataset using the defined profiling rules and display results
validation_results = validate_dataset(df, default_profiling_rules)
print("Validation Results:", validation_results)

Validation Results: {'Customer_ID': 'Valid', 'Account_Balance': 'Valid', 'Transaction_Amount': 'Valid', 'Reported_Amount': 'Valid', 'Currency': 'Valid', 'Country': "Column 'Country' has invalid values at rows: [2]", 'Transaction_Date': 'Valid', 'Risk_Score': 'Valid', 'Transaction_Amount vs Reported_Amount': 'Cross-field validation failed at rows: [3]', 'Round_Number_Transaction': 'Cross-field validation failed at rows: [3]', 'High_Risk_Transaction': 'Valid'}


In [9]:
# Detect anomalies in the dataset and display the results
df = detect_anomalies(df)
print("Anomaly Detection Results:\n", df[['Customer_ID', 'Anomaly']])

Anomaly Detection Results:
    Customer_ID  Anomaly
0         1001        1
1         1002        1
2         1003        1
3         1004       -1


In [10]:
# Apply adaptive risk scoring to the dataset and display the results
df['Adaptive_Risk_Score'] = df.apply(lambda row: adjust_risk_score(row, validation_results), axis=1)
print("Adaptive Risk Scores:\n", df[['Customer_ID', 'Risk_Score', 'Adaptive_Risk_Score', 'Anomaly']])

Adaptive Risk Scores:
    Customer_ID  Risk_Score  Adaptive_Risk_Score  Anomaly
0         1001         3.0                  3.0        1
1         1002         2.0                  2.0        1
2         1003         NaN                  4.0        1
3         1004         5.0                 12.0       -1


In [11]:
# Generate remediation suggestions for flagged transactions with detailed explanations
def suggest_remediation(row, validation_results):
    suggestions = []
    for column, result in validation_results.items():
        if 'rows' in str(result):
            try:
                invalid_rows = [int(r) for r in str(result).split('[')[1].split(']')[0].split(',') if r.strip().isdigit()]
                if row.name in invalid_rows:
                    if column == 'Transaction_Amount vs Reported_Amount':
                        suggestions.append(f"Adjust Reported_Amount to match Transaction_Amount within 1% deviation if cross-currency (Current: Transaction_Amount={row['Transaction_Amount']}, Reported_Amount={row['Reported_Amount']}).")
                    elif column == 'Currency':
                        suggestions.append("Correct Currency to a valid ISO 4217 code (e.g., USD, EUR, GBP, JPY).")
                    elif column == 'Country':
                        suggestions.append("Correct Country to a valid jurisdiction (e.g., US, DE, UK, JP).")
                    elif column == 'Transaction_Date':
                        suggestions.append("Review Transaction_Date; ensure it’s not future-dated or over 365 days old.")
                    elif column == 'Round_Number_Transaction':
                        suggestions.append(f"Review transaction for potential money laundering risk due to round number (Transaction_Amount={row['Transaction_Amount']}).")
            except (IndexError, ValueError):
                pass
    if row['Anomaly'] == -1:
        suggestions.append("Review transaction for potential anomalies in Account_Balance, Transaction_Amount, or Reported_Amount.")
    return suggestions

df['Remediation_Suggestions'] = df.apply(lambda row: suggest_remediation(row, validation_results), axis=1)
print("Remediation Suggestions:\n", df[['Customer_ID', 'Remediation_Suggestions']])

Remediation Suggestions:
    Customer_ID                            Remediation_Suggestions
0         1001                                                 []
1         1002                                                 []
2         1003  [Correct Country to a valid jurisdiction (e.g....
3         1004  [Adjust Reported_Amount to match Transaction_A...


In [ ]:
# Use an LLM to generate profiling rules from regulatory instructions, with a fallback manual implementation
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load StarCoder with 8-bit quantization for CPU usage
checkpoint = "bigcode/starcoder"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
from transformers import BitsAndBytesConfig

# Load model without 8-bit quantization for CPU
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    device_map={"": "cpu"},  # Force CPU usage
    torch_dtype=torch.float32  # Use float32 since FP16 may not work on CPU
)

generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    device=-1  # -1 for CPU (since your device is set to CPU)
)

instructions = """
Transaction_Amount should always match Reported_Amount, except when the transaction involves cross-currency conversions, in which case a permissible deviation of up to 1% is allowed.
Currency should be a valid ISO 4217 currency code.
"""

prompt = f"Convert these regulatory instructions into Python validation functions:\n\n{instructions}\n\nExample:\ndef validate_amounts(row):\n    if row['Cross_Currency'] == 'No':\n        return row['Transaction_Amount'] == row['Reported_Amount']\n    else:\n        return abs(row['Transaction_Amount'] - row['Reported_Amount']) <= 0.01 * row['Transaction_Amount']\n\nProvide similar Python functions for the given instructions."

try:
    # Use max_new_tokens to specify the number of new tokens to generate, excluding the input prompt
    generated_rules = generator(prompt, max_new_tokens=100)[0]['generated_text']
    print("Generated Rules:\n", generated_rules)
    # Create the artifacts directory if it doesn't exist
    os.makedirs('artifacts', exist_ok=True)
    # Save the generated rules to a file with a timestamp for documentation
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    rules_filename = f'artifacts/generated_rules_{timestamp}.txt'
    try:
        with open(rules_filename, 'w') as f:
            f.write(f"# Generated Rules\n# Timestamp: {timestamp}\n# Prompt:\n{prompt}\n\n")
            f.write(generated_rules)
        print(f"Generated rules saved to '{rules_filename}' for documentation.")
    except Exception as e:
        print(f"Failed to save generated rules to '{rules_filename}': {e}")
    # Check if the generated output contains a complete Python function definition
    if "def validate_" in generated_rules and "return" in generated_rules and generated_rules.count('def') == generated_rules.count('return'):
        print("Generated rules appear to contain complete Python functions. You can parse and use them.")
        summary_status = "Usable"
        summary_filenames = f"Generated: {rules_filename}"
    else:
        print("Generated rules are not usable Python functions (incomplete or incorrect syntax). Using manual implementation instead.")
        manual_rules = """
def validate_amounts(row):
    if row.get('Cross_Currency', 'No') == 'No':
        return row['Transaction_Amount'] == row['Reported_Amount']
    return abs(row['Transaction_Amount'] - row['Reported_Amount']) <= 0.01 * row['Transaction_Amount']

def validate_currency(row):
    iso_4217_codes = {'USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'CNY', 'INR', 'SGD'}  # Expanded list
    return row['Currency'] in iso_4217_codes
        """
        print("Manual Rules (integrated into Cell 2):\n", manual_rules)
        # Save the manual rules to a file for documentation
        manual_rules_filename = f'artifacts/manual_rules_{timestamp}.txt'
        try:
            with open(manual_rules_filename, 'w') as f:
                f.write(f"# Manual Rules (Fallback)\n# Timestamp: {timestamp}\n# Used because generated rules were unusable\n\n")
                f.write(manual_rules)
            print(f"Manual rules saved to '{manual_rules_filename}' for documentation.")
        except Exception as e:
            print(f"Failed to save manual rules to '{manual_rules_filename}': {e}")
        summary_status = "Unusable (used manual rules)"
        summary_filenames = f"Generated: {rules_filename}, Manual: {manual_rules_filename}"
except Exception as e:
    print(f"LLM generation failed: {e}")
    # Fallback: Manually implement the rules if LLM fails
    print("Using manually implemented rules as fallback (integrated into Cell 2):")
    manual_rules = """
def validate_amounts(row):
    if row.get('Cross_Currency', 'No') == 'No':
        return row['Transaction_Amount'] == row['Reported_Amount']
    return abs(row['Transaction_Amount'] - row['Reported_Amount']) <= 0.01 * row['Transaction_Amount']

def validate_currency(row):
    iso_4217_codes = {'USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'CNY', 'INR', 'SGD'}  # Expanded list
    return row['Currency'] in iso_4217_codes
    """
    print(manual_rules)
    # Save the manual rules to a file for documentation
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    manual_rules_filename = f'artifacts/manual_rules_{timestamp}.txt'
    try:
        with open(manual_rules_filename, 'w') as f:
            f.write(f"# Manual Rules (Fallback)\n# Timestamp: {timestamp}\n# Used because LLM generation failed\n\n")
            f.write(manual_rules)
        print(f"Manual rules saved to '{manual_rules_filename}' for documentation.")
    except Exception as e:
        print(f"Failed to save manual rules to '{manual_rules_filename}': {e}")
    summary_status = "LLM Failed (used manual rules)"
    summary_filenames = f"Manual: {manual_rules_filename}"

# Append a summary of this run to a log file
summary_log_filename = 'artifacts/summary_log.txt'
try:
    with open(summary_log_filename, 'a') as f:
        f.write(f"Run at {timestamp}: Status: {summary_status}, Files: {summary_filenames}\n")
    print(f"Summary logged to '{summary_log_filename}'.")
except Exception as e:
    print(f"Failed to log summary to '{summary_log_filename}': {e}")

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00007-of-00007.safetensors:  75%|#######5  | 3.08G/4.08G [00:00<?, ?B/s]

model-00001-of-00007.safetensors:  30%|###       | 2.99G/9.90G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:  24%|##3       | 2.33G/9.86G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 41.9M/9.85G [00:00<?, ?B/s]

...

Error while downloading from https://cdn-lfs.hf.co/repos/8a/c7/8ac75c6365389b6ca1b539464ff5fce142c1315639eec6f7f29cc93c9d063b0a/66c0e28284d699d79e61adde2db19924ac1bf0b20584dadb33c36abe21c5d425?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00006-of-00007.safetensors%3B+filename%3D%22model-00006-of-00007.safetensors%22%3B&Expires=1743002743&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzAwMjc0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy84YS9jNy84YWM3NWM2MzY1Mzg5YjZjYTFiNTM5NDY0ZmY1ZmNlMTQyYzEzMTU2MzllZWM2ZjdmMjljYzkzYzlkMDYzYjBhLzY2YzBlMjgyODRkNjk5ZDc5ZTYxYWRkZTJkYjE5OTI0YWMxYmYwYjIwNTg0ZGFkYjMzYzM2YWJlMjFjNWQ0MjU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=K9N7gg-YZOZt8g5E7Rq31PriH1VyVwxqL%7EnqwTfKVq4WyPQlJohxW3qE%7ELNQ42EA5tHPrB84EAY9JxTT8RdQfQLI8YTaxpxViJzAny4VFS8m3wqw74KnlO0abasRCSBtWSh%7E3xMxRmhrSbuYvF9ay6W2X-O%7E64dkVH06IPT8wDQJDEu%7EOhkmAvc8hrxrehxnAyvs1z6wh3A%7Ev-eRFH2QAvDr%7EwPqFwj

model-00001-of-00007.safetensors:  34%|###4      | 3.41G/9.90G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/8a/c7/8ac75c6365389b6ca1b539464ff5fce142c1315639eec6f7f29cc93c9d063b0a/08dc737cea124916697803456dffbe63234705c7e41eee411410a62ae0e3e6dd?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00007.safetensors%3B+filename%3D%22model-00001-of-00007.safetensors%22%3B&Expires=1743002743&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzAwMjc0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy84YS9jNy84YWM3NWM2MzY1Mzg5YjZjYTFiNTM5NDY0ZmY1ZmNlMTQyYzEzMTU2MzllZWM2ZjdmMjljYzkzYzlkMDYzYjBhLzA4ZGM3MzdjZWExMjQ5MTY2OTc4MDM0NTZkZmZiZTYzMjM0NzA1YzdlNDFlZWU0MTE0MTBhNjJhZTBlM2U2ZGQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=jPPKFoDmt%7Ecpi034Ujght7xmPB235T-7T-04rFj8eT%7E7dnm2nr1xXFmRAKq65YTT92xhP75jZ3rsSj7wzuGsCCFhZdhSQEhaZzZzZSuQpGEtk3M1XC3MLauXHnIFbKVfymFfTOmunDhLf7aFXf7uXLiKqpVPzmmozNxwCMbZDVeLYHH7ELHJ9HVVdPuuuHCyzfQHMGG9dIUT1yFLoTAoma1Xj6100EPy5CvRXxKJv

model-00001-of-00007.safetensors:  61%|######1   | 6.05G/9.90G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/8a/c7/8ac75c6365389b6ca1b539464ff5fce142c1315639eec6f7f29cc93c9d063b0a/00eb327bb77018f0d9df9332faa904f9dcc9b1c9e2f7fe981dec4af5b808dffb?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00007.safetensors%3B+filename%3D%22model-00002-of-00007.safetensors%22%3B&Expires=1743002743&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzAwMjc0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy84YS9jNy84YWM3NWM2MzY1Mzg5YjZjYTFiNTM5NDY0ZmY1ZmNlMTQyYzEzMTU2MzllZWM2ZjdmMjljYzkzYzlkMDYzYjBhLzAwZWIzMjdiYjc3MDE4ZjBkOWRmOTMzMmZhYTkwNGY5ZGNjOWIxYzllMmY3ZmU5ODFkZWM0YWY1YjgwOGRmZmI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=dFZHVshqIc7leAjN6LBxLtXrGvrFXSZ4389sD%7E4v19efFGXPB4n0g1-ou%7E2IUoVQK0n%7El35ylK4rt3EEHvU8llN4jraO4hOsVLQoBhqsAbEMFNv2YCnd23tfJLrdqlmnWMBdVwyfGHETGieUxPh24ziAWese%7EyAksZzLaNcZmTcbcXmhnSOo9pihYPZJSFoVG8UmwEKbpGERPvGTVHUd5SFu-ejD1IQ0q9vvz